# Decentralized Multi-Robot Path Planning System Using Graph Neural Network (GNN)

## Importing the libraries

In [1]:
from Dataset_Generator import DatasetGenerator
from pre_processing import Preprocessing
from Adjacency_Matrix import adj_mat
from Decentralized_Model import decentralizedModel
from Test_Model import TestModel
import torch
import numpy as np
import random
import torch.nn as nn
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU setups
grid_size = 20
num_of_robots = 10
rfov = 3
# Optional: Force deterministic behavior on GPU (may impact performance)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Dataset Generation

In [ ]:
grid = [[0 for _ in range(grid_size)] for _ in range(grid_size)]  # Generate grid map
dataset_generator = DatasetGenerator(num_cases=10000, num_agents=num_of_robots, grid=grid)
# cases = dataset_generator.generate_cases()
# dataset_generator.save_cases_to_file(cases, "dataset.json")
# print(f"Generated and saved {len(cases)} cases.")
cases = dataset_generator.load_cases_from_file("dataset.json")

## Preprocessing

In [ ]:
p = Preprocessing(grid,cases, rfov)
data_tensors  = p.begin()

#### Adjacency Matrix

In [ ]:
adj = adj_mat(cases,rfov)
adj_matrices = adj.get_adj_mat()

## Model Training

In [ ]:
trained_model = decentralizedModel(cases, data_tensors,rfov = rfov,adjacency_matrix = adj_matrices,num_of_robots = num_of_robots, k_hops = 2, num_epochs = 40, checkpoint_path='dataset_10000_on_10_robots.pth')

In [ ]:
if torch.cuda.is_available():
    trained_model.to(device="cuda")

In [ ]:
trained_model.train_model()

## Model Evaluation

In [ ]:
trained_model.test_model()

Test Loss: 0.0837, Test Accuracy: 97.62%


(0.0836656704364765, 0.9762399193548387)

In [3]:
num_of_cases = 1000
test_model = TestModel(num_of_robots= 8, num_of_cases= num_of_cases, grid_size= grid_size, model_pth= 'dataset_10000_on_10_robots.pth',random_seed=23, rfov = rfov)
reached, all_confilicts, all_steps, conflict_set_cases = test_model.test()
not_reached = num_of_cases - reached
success_rate = ((num_of_cases - (not_reached + len(conflict_set_cases))) / num_of_cases)*100
print(f"success rate = {success_rate}%")

case  0
case added
reached cases = 999
all confilicts = 168
all steps = 23736
cases with confilict = 117
success rate = 88.2%
